In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.v2 as transforms_v2
import sys
import os
import time
import random
import numpy as np
import wandb
from tqdm import tqdm
from tools.tool import AccuracyEarlyStopping, WarmUpLR, SAM
# Wide ResNet 모델 임포트
from models.wideresidual import Wide_ResNet, conv_init

# 중요: wideresidual.py 파일에서 "*wide*layer" 메서드명을 "_wide_layer"로 수정해야 합니다!

wandb.login(key="ef091b9abcea3186341ddf8995d62bde62d7469e")
wandb.init(project="PBL-2", name="wideresnet_28_20_cfc,SAM_SGD")  

# WandB 설정 - WideResNet 28x20으로 변경
config = {
    "model": "wideresnet",  
    "depth": 28,            
    "widen_factor": 20,     # 10에서 20으로 변경
    "dropout_rate": 0.3,    
    "batch_size": 128,       # 메모리 부족 가능성 고려하여 더 작은 배치 크기로 조정
    "num_epochs": 300,
    "learning_rate": 0.01,  
    "optimizer": "SGD",     
    "momentum": 0.9,        
    "weight_decay": 5e-4,   
    "deterministic": False,
    "patience": 30,         
    "max_epochs_wait": float('inf'),
    "cutmix_alpha": 1.0,    
    "cutmix_prob": 0.5,     
    "crop_padding": 4,      
    "crop_size": 32,        
    "warmup_epochs": 5,     
}
wandb.config.update(config)


# CIFAR-100 데이터셋 로드 - 강화된 데이터 증강 추가
transform_train = transforms.Compose([
    transforms.RandomCrop(config["crop_size"], padding=config["crop_padding"]),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

trainset = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform_train)

testset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform_test)

# DataLoader 생성
trainloader = DataLoader(trainset, batch_size=config["batch_size"], shuffle=True, 
                        num_workers=32, pin_memory=True)  # GPU 전송 속도 향상
testloader = DataLoader(testset, batch_size=config["batch_size"], shuffle=False, 
                       num_workers=32, pin_memory=True)

print(f"Train set size: {len(trainset)}")
print(f"Test set size: {len(testset)}")

# CutMix 변환 정의
cutmix = transforms_v2.CutMix(alpha=config["cutmix_alpha"], num_classes=100)  # CIFAR-100은 100개 클래스

def train(model, trainloader, criterion, optimizer, device, epoch, warmup_scheduler=None, warmup_epochs=None):
    """
    학습 함수 (CutMix 적용)
    """
    if warmup_epochs is None:
        warmup_epochs = config["warmup_epochs"]
        
    model.train()   # 모델을 학습 모드로 설정
    start_time = time.time()  # 시간 측정 시작
    running_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # CutMix 확률적 적용
        if random.random() < config["cutmix_prob"]:
            inputs, labels = cutmix(inputs, labels)
            use_cutmix = True
        else:
            use_cutmix = False
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # CutMix 적용 여부에 따라 손실 함수 선택
        if use_cutmix:
            loss = torch.nn.functional.cross_entropy(outputs, labels)
        else:
            loss = criterion(outputs, labels)
            
        loss.backward()
        optimizer.first_step(zero_grad=True)
        
        # 두 번째 forward-backward 패스 (SAM 최적화)
        outputs = model(inputs)  # 항상 새 forward 패스 필요
        
        if use_cutmix:
            loss = torch.nn.functional.cross_entropy(outputs, labels)
        else:
            loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.second_step(zero_grad=True)
        
        # 학습률 스케줄러 업데이트
        if epoch < warmup_epochs and warmup_scheduler is not None:
            warmup_scheduler.step()
        
        running_loss += loss.item()

        # 정확도 계산
        if use_cutmix:
            _, label_idx = labels.max(1)
        else:
            label_idx = labels
        
        # top-1 정확도 계산
        _, predicted = outputs.max(1)
        total += inputs.size(0)
        correct_top1 += predicted.eq(label_idx).sum().item()
        
        # top-5 정확도 계산
        _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
        correct_top5 += sum([1 for i in range(len(label_idx)) if label_idx[i] in top5_idx[i]])
        
        if (i + 1) % 50 == 0:  # 50 배치마다 출력
            print(f'Epoch [{epoch+1}], Batch [{i+1}/{len(trainloader)}], Loss: {loss.item():.4f}, LR: {optimizer.param_groups[0]["lr"]:.6f}')
    
    epoch_loss = running_loss / len(trainloader)
    accuracy_top1 = 100.0 * correct_top1 / total
    accuracy_top5 = 100.0 * correct_top5 / total
    
    train_time = time.time() - start_time
    
    # 학습 세트에 대한 성능 출력
    print(f'Train set: Epoch: {epoch+1}, Average loss:{epoch_loss:.4f}, LR: {optimizer.param_groups[0]["lr"]:.6f} '
          f'Top-1 Accuracy: {accuracy_top1:.4f}%, Top-5 Accuracy: {accuracy_top5:.4f}%, Time consumed:{train_time:.2f}s')
    
    return epoch_loss, accuracy_top1, accuracy_top5

def evaluate(model, dataloader, criterion, device, epoch, phase="test"):
    """
    평가 함수
    """
    model.eval()  # 모델을 평가 모드로 설정
    start_time = time.time()  # 시간 측정 시작
    
    eval_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    
    # 그래디언트 계산 비활성화
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # 순전파
            outputs = model(inputs)
            
            # 손실 계산
            loss = criterion(outputs, labels)
            eval_loss += loss.item()
            
            # top-1 정확도 계산
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct_top1 += (predicted == labels).sum().item()
            
            # top-5 정확도 계산
            _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
            correct_top5 += top5_idx.eq(labels.view(-1, 1).expand_as(top5_idx)).sum().item()
    
    # 평균 손실 및 정확도 계산
    eval_loss = eval_loss / len(dataloader)
    accuracy_top1 = 100.0 * correct_top1 / total
    accuracy_top5 = 100.0 * correct_top5 / total
    
    # 평가 시간 계산
    eval_time = time.time() - start_time
    
    # 테스트 세트에 대한 성능 출력
    print(f'{phase.capitalize()} set: Epoch: {epoch+1}, Average loss:{eval_loss:.4f}, '
          f'Top-1 Accuracy: {accuracy_top1:.4f}%, Top-5 Accuracy: {accuracy_top5:.4f}%, Time consumed:{eval_time:.2f}s')
    print()
    
    return eval_loss, accuracy_top1, accuracy_top5

def main_training_loop(model, trainloader, testloader, criterion, optimizer, device, num_epochs, patience, max_epochs_wait, warmup_scheduler=None, main_scheduler=None, warmup_epochs=None):
    """
    메인 학습 루프 (accuracy 기준 early stopping)
    """
    if warmup_epochs is None:
        warmup_epochs = config["warmup_epochs"]
        
    # 정확도 기반 얼리 스토핑 사용
    early_stopping = AccuracyEarlyStopping(patience=patience, verbose=True, path='checkpoint.pt', max_epochs=max_epochs_wait)
    
    best_test_acc_top1 = 0.0
    best_test_acc_top5 = 0.0
    
    # 테스트 정확도 기록을 위한 리스트
    test_acc_top1_history = []
    
    # tqdm을 사용한 진행 상황 표시
    for epoch in tqdm(range(num_epochs)):
        # 학습
        train_loss, train_acc_top1, train_acc_top5 = train(
            model, 
            trainloader, 
            criterion, 
            optimizer, 
            device, 
            epoch, 
            warmup_scheduler, 
            warmup_epochs
        )
        
        # 테스트 데이터로 평가
        test_loss, test_acc_top1, test_acc_top5 = evaluate(model, testloader, criterion, device, epoch, phase="test")

        # 웜업 이후 ReduceLROnPlateau 스케줄러 업데이트 
        if epoch >= warmup_epochs and main_scheduler is not None:
            main_scheduler.step(test_acc_top1)  # 테스트 정확도에 따라 학습률 업데이트       
            
        # 테스트 정확도 기록
        test_acc_top1_history.append(test_acc_top1)
        
        # WandB에 로깅
        wandb.log({
            "epoch": epoch + 1,
            "learning_rate": optimizer.param_groups[0]['lr'],
            "train_loss": train_loss,
            "train_accuracy_top1": train_acc_top1,
            "train_accuracy_top5": train_acc_top5,
            "test_loss": test_loss,
            "test_accuracy_top1": test_acc_top1,
            "test_accuracy_top5": test_acc_top5
        })
            
        # 최고 정확도 모델 저장 (top-1 기준)
        if test_acc_top1 > best_test_acc_top1:
            best_test_acc_top1 = test_acc_top1
            best_test_acc_top5_at_best_top1 = test_acc_top5
            print(f'새로운 최고 top-1 정확도: {best_test_acc_top1:.2f}%, top-5 정확도: {best_test_acc_top5_at_best_top1:.2f}%')
            # 모델 저장
            model_path = f'best_model_{wandb.run.name}.pth'
            if isinstance(model, nn.DataParallel):
                torch.save(model.module.state_dict(), model_path)  # DataParallel 래퍼 제거
            else:
                torch.save(model.state_dict(), model_path)
            
            # WandB에 모델 아티팩트 저장
            wandb.save(model_path)
        
        # top-5 accuracy 기록 업데이트
        if test_acc_top5 > best_test_acc_top5:
            best_test_acc_top5 = test_acc_top5
            print(f'새로운 최고 top-5 정확도: {best_test_acc_top5:.2f}%')

        # Early stopping 체크 (test_acc_top1 기준)
        early_stopping(test_acc_top1, model, epoch)
        if early_stopping.early_stop:
            print(f"에폭 {epoch+1}에서 학습 조기 종료. 최고 성능 에폭: {early_stopping.best_epoch+1}")
            break
    
    # 훈련 완료 후 최고 모델 로드
    print("테스트 정확도 기준 최고 모델 로드 중...")
    model_path = f'best_model_{wandb.run.name}.pth'
    
    # 올바른 모델 로드를 위한 처리
    if isinstance(model, nn.DataParallel):
        model.module.load_state_dict(torch.load(model_path))
    else:
        model.load_state_dict(torch.load(model_path))

    # 최종 테스트 세트 평가
    final_test_loss, final_test_acc_top1, final_test_acc_top5 = evaluate(model, testloader, criterion, device, num_epochs-1, phase="test")
    
    print(f'완료! 최고 테스트 top-1 정확도: {best_test_acc_top1:.2f}%, 최고 테스트 top-5 정확도: {best_test_acc_top5:.2f}%')
    print(f'최종 테스트 top-1 정확도: {final_test_acc_top1:.2f}%, 최종 테스트 top-5 정확도: {final_test_acc_top5:.2f}%')
    
    # WandB에 최종 결과 기록
    wandb.run.summary["best_test_accuracy_top1"] = best_test_acc_top1
    wandb.run.summary["best_test_accuracy_top5"] = best_test_acc_top5
    wandb.run.summary["final_test_accuracy_top1"] = final_test_acc_top1
    wandb.run.summary["final_test_accuracy_top5"] = final_test_acc_top5

    # Early stopping 정보 저장
    if early_stopping.early_stop:
        wandb.run.summary["early_stopped"] = True
        wandb.run.summary["early_stopped_epoch"] = epoch+1
        wandb.run.summary["best_epoch"] = early_stopping.best_epoch+1
    else:
        wandb.run.summary["early_stopped"] = False


# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 메모리 사용량 출력 함수 (디버깅용)
def print_gpu_memory():
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
            print(f"Memory Allocated: {torch.cuda.memory_allocated(i)/1024**3:.2f} GB")
            print(f"Memory Reserved: {torch.cuda.memory_reserved(i)/1024**3:.2f} GB")
            print(f"Max Memory Allocated: {torch.cuda.max_memory_allocated(i)/1024**3:.2f} GB")

# GPU 메모리 초기 상태 출력
print_gpu_memory()

# Wide ResNet 모델 초기화
model = Wide_ResNet(
    depth=config["depth"],
    widen_factor=config["widen_factor"],
    dropout_rate=config["dropout_rate"],
    num_classes=100  # CIFAR-100은 100개 클래스
).to(device)

# 가중치 초기화 적용
model.apply(conv_init)

# 모델 파라미터 수 계산 및 출력
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

num_params = count_parameters(model)
print(f'Wide-ResNet-{config["depth"]}-{config["widen_factor"]} initialized with {num_params:,} parameters')
wandb.run.summary["model_parameters"] = num_params

# 손실 함수 설정 (라벨 스무딩 사용)
criterion = nn.CrossEntropyLoss()

# SGD 옵티마이저 사용
base_optimizer = optim.SGD
optimizer = SAM(
    model.parameters(), 
    base_optimizer, 
    lr=config["learning_rate"],
    momentum=config["momentum"],
    weight_decay=config["weight_decay"],
    nesterov=True  # Nesterov 모멘텀 사용
)

# WarmUpLR 스케줄러 초기화
warmup_steps = config["warmup_epochs"] * len(trainloader)
warmup_scheduler = WarmUpLR(optimizer, total_iters=warmup_steps)

# 웜업 이후 사용할 스케줄러 설정 
main_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='max',          
    factor=0.5,          
    patience=5,          
    verbose=True,        
    threshold=0.01,      
    min_lr=1e-6
)

# WandB에 모델 구조 기록
wandb.watch(model, log="all")

# GPU 가속
if torch.cuda.device_count() > 1:
    print(f"{torch.cuda.device_count()}개의 GPU를 사용합니다.")
    model = nn.DataParallel(model)
    
# 모델 상태 요약 출력
print(f"Model: WideResNet-{config['depth']}-{config['widen_factor']}")
print(f"Batch size: {config['batch_size']}")
print(f"Learning rate: {config['learning_rate']}")
print(f"Optimizer: {config['optimizer']} with SAM")
print(f"Dropout rate: {config['dropout_rate']}")

# 훈련 시작 시간 기록
start_time = time.time()

try:
    # 메인 학습 루프 호출
    main_training_loop(
        model=model,
        trainloader=trainloader,
        testloader=testloader,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        num_epochs=config["num_epochs"],
        patience=config["patience"],
        max_epochs_wait=config["max_epochs_wait"],
        warmup_scheduler=warmup_scheduler,
        main_scheduler=main_scheduler,
        warmup_epochs=config["warmup_epochs"]
    )
except Exception as e:
    print(f"학습 중 오류 발생: {e}")
    # 오류 발생시 메모리 상태 출력
    print_gpu_memory()
    raise

# 훈련 종료 시간 기록 및 출력
end_time = time.time()
total_time = end_time - start_time
wandb.log({"total_training_time": total_time})

print(f"전체 학습 시간: {total_time:.2f} 초")
print(f"전체 학습 시간: {total_time/3600:.2f} 시간")

# 최종 메모리 상태 출력
print_gpu_memory()

# WandB 실행 종료
wandb.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


model_parameters,36546980


Files already downloaded and verified
Files already downloaded and verified
Train set size: 50000
Test set size: 10000
Using device: cuda
GPU 0: NVIDIA RTX A5000
Memory Allocated: 0.29 GB
Memory Reserved: 4.14 GB
Max Memory Allocated: 3.67 GB
GPU 1: NVIDIA RTX A5000
Memory Allocated: 0.02 GB
Memory Reserved: 3.75 GB
Max Memory Allocated: 3.08 GB
| Wide-Resnet 28x20


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Wide-ResNet-28-20 initialized with 145,950,980 parameters
2개의 GPU를 사용합니다.
Model: WideResNet-28-20
Batch size: 128
Learning rate: 0.01
Optimizer: SGD with SAM
Dropout rate: 0.3


...

Test set: Epoch: 117, Average loss:1.3534, Top-1 Accuracy: 73.0300%, Top-5 Accuracy: 92.3000%, Time consumed:14.90s

EarlyStopping 카운터: 29 / 30
Epoch [118], Batch [50/391], Loss: 0.1086, LR: 0.000010
Epoch [118], Batch [100/391], Loss: 0.1267, LR: 0.000010
Epoch [118], Batch [150/391], Loss: 1.5958, LR: 0.000010
Epoch [118], Batch [200/391], Loss: 0.1023, LR: 0.000010
Epoch [118], Batch [250/391], Loss: 0.1167, LR: 0.000010
Epoch [118], Batch [300/391], Loss: 0.0895, LR: 0.000010
Epoch [118], Batch [350/391], Loss: 1.9541, LR: 0.000010
Train set: Epoch: 118, Average loss:0.9796, LR: 0.000010 Top-1 Accuracy: 85.1340%, Top-5 Accuracy: 96.2360%, Time consumed:352.55s


 39%|█████████████████████████████████▌                                                    | 117/300 [12:05:03<18:54:04, 371.83s/it]

Test set: Epoch: 118, Average loss:1.3297, Top-1 Accuracy: 72.4300%, Top-5 Accuracy: 91.9300%, Time consumed:14.55s

EarlyStopping 카운터: 30 / 30
에폭 118에서 학습 조기 종료. 최고 성능 에폭: 88
테스트 정확도 기준 최고 모델 로드 중...


Test set: Epoch: 300, Average loss:1.2777, Top-1 Accuracy: 73.6600%, Top-5 Accuracy: 92.2900%, Time consumed:14.60s

완료! 최고 테스트 top-1 정확도: 73.66%, 최고 테스트 top-5 정확도: 92.50%
최종 테스트 top-1 정확도: 73.66%, 최종 테스트 top-5 정확도: 92.29%
전체 학습 시간: 43519.32 초
전체 학습 시간: 12.09 시간
GPU 0: NVIDIA RTX A5000
Memory Allocated: 1.38 GB
Memory Reserved: 7.20 GB
Max Memory Allocated: 5.14 GB
GPU 1: NVIDIA RTX A5000
Memory Allocated: 0.02 GB
Memory Reserved: 5.25 GB
Max Memory Allocated: 3.44 GB


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
learning_rate,▂███████████████▄▄▃▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy_top1,▁▁▁▁▃▃▄▄▄▄▅▅▅▆▆▆▇▇▆▇▇▇▇█████████████████
test_accuracy_top5,▁▂▂▃▄▅▅▆▆▇▇▇▇▇▇▇████████████████████████
test_loss,█▆▆▄▃▃▂▃▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_training_time,▁
train_accuracy_top1,▁▂▂▃▃▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇███████████████
train_accuracy_top5,▁▃▄▅▅▆▆▇▆▇▇▇▇▇▇▇████████████████████████
train_loss,█▇▆▆▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,88
best_test_accuracy_top1,73.66
